In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import sqlite3

In [2]:
# Load the dataset
file_path = "D:/DE_final_projects/dataset/hotel_booking_web.csv"
df = pd.read_csv(file_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5262 entries, 0 to 5261
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   State             5262 non-null   object 
 1   Cities            5262 non-null   object 
 2   Hotel Name        5262 non-null   object 
 3   Destination       5262 non-null   object 
 4   Describe          5262 non-null   object 
 5   Ratings           5262 non-null   float64
 6   review_count      5262 non-null   int64  
 7   Actual Price      5262 non-null   int64  
 8   Offer Percentage  5262 non-null   int64  
 9   Price             5262 non-null   int64  
dtypes: float64(1), int64(4), object(5)
memory usage: 411.2+ KB


In [ ]:
# Summary statistics for numerical columns
summary_stats = df.describe()
summary_stats

# Check unique values in categorical columns
categorical_summary = {col: df[col].nunique() for col in df.select_dtypes(include=['object']).columns}
categorical_summary

(           Ratings  review_count  Actual Price  Offer Percentage        Price
 count  5262.000000   5262.000000   5262.000000       5262.000000  5262.000000
 mean      3.979666    250.819460   4102.655454         70.971114   966.178259
 std       0.565988    443.599321   2354.831276          3.626750   626.394972
 min       3.000000      1.000000   1442.000000         29.000000   281.000000
 25%       3.500000     16.000000   2824.000000         68.000000   619.000000
 50%       4.000000     80.000000   3604.500000         71.000000   823.000000
 75%       4.500000    294.000000   4623.500000         72.000000  1108.000000
 max       5.000000   6540.000000  39532.000000         92.000000  9392.000000,
 {'State': 28,
  'Cities': 151,
  'Hotel Name': 4746,
  'Destination': 4185,
  'Describe': 5})

In [13]:
# Create unique States table with IDs
states_df = pd.DataFrame(df["State"].unique(), columns=["State"])
states_df["State_ID"] = range(1, len(states_df) + 1)

# Create unique Cities table with IDs
cities_df = pd.DataFrame(df["Cities"].unique(), columns=["City"])
cities_df["City_ID"] = range(1, len(cities_df) + 1)

# Merge to get State_ID and City_ID in the main table
df = df.merge(states_df, on="State", how="left")
df = df.merge(cities_df, left_on="Cities", right_on="City", how="left")

# Drop original State and City columns
df_main = df.drop(columns=["State", "Cities", "City"])



In [14]:
# Save as three separate database files
states_db_path = "D:/DE_final_projects/dataset/state.csv"
cities_db_path = "D:/DE_final_projects/dataset/cities.csv"
main_db_path = "D:/DE_final_projects/dataset/main.csv"

states_df.to_csv(states_db_path, index=False)
cities_df.to_csv(cities_db_path, index=False)
df_main.to_csv(main_db_path, index=False)

(states_db_path, cities_db_path, main_db_path)


('D:/DE_final_projects/dataset/state.csv',
 'D:/DE_final_projects/dataset/cities.csv',
 'D:/DE_final_projects/dataset/main.csv')

In [15]:

# Define database file path
db_file_path = "D:/DE_final_projects/dataset/hotel_web.db"

# Create SQLite database and save tables
conn = sqlite3.connect(db_file_path)

# Save DataFrames as tables in SQLite
states_df.to_sql("states", conn, index=False, if_exists="replace")
cities_df.to_sql("cities", conn, index=False, if_exists="replace")
df_main.to_sql("hotel_bookings", conn, index=False, if_exists="replace")



5262

In [16]:
# Close connection
conn.close()



In [17]:

# Reconnect to the SQLite database
conn = sqlite3.connect(db_file_path)


In [18]:

# Load data into DataFrames
df_hotels = pd.read_sql("SELECT * FROM hotel_bookings", conn)
df_states = pd.read_sql("SELECT * FROM states", conn)
df_cities = pd.read_sql("SELECT * FROM cities", conn)




In [19]:
# Merge tables for meaningful analysis
df_hotels = df_hotels.merge(df_states, on="State_ID", how="left")
df_hotels = df_hotels.merge(df_cities, on="City_ID", how="left")


In [20]:

# Close connection
conn.close()

In [21]:
df_hotels


,Hotel Name,Destination,Describe,Ratings,review_count,Actual Price,Offer Percentage,Price,State_ID,City_ID,State,City
0,Hotel O Sai President,Near Rawali Mandir,Good,3.5,453,3962,71,932,1,1,Uttar Pradesh,agra
1,Hotel O ASA AGRA,Plot number 1 Fatehabad road Plot number 1 Fat...,Excellent,4.5,275,2912,72,648,1,1,Uttar Pradesh,agra
2,Hotel O Salt And Pepper Hotel And Resturant,Near Prachin Pathwari Mandir,Good,3.5,137,4948,71,1137,1,1,Uttar Pradesh,agra
3,Hotel O KING PALACE,Near Fatehabad Road,Excellent,4.5,9,3378,71,797,1,1,Uttar Pradesh,agra
4,Hotel O 804621 Hotel Vintage House,1717 Fatehabad Road,Excellent,4.5,134,2912,70,702,1,1,Uttar Pradesh,agra
...,...,...,...,...,...,...,...,...,...,...,...,...
5257,Hotel O Elista,India - 140603,Very Good,4.0,138,4051,68,1036,5,151,Punjab,zirakpur
5258,Hotel O AV Continental,Near Chandigarh Airport Next to New Life Line ...,Fair,3.0,50,1797,62,542,5,151,Punjab,zirakpur
5259,Hotel O The Banur Heights,Mohali,Good,3.5,35,6552,77,1212,5,151,Punjab,zirakpur
5260,Collection O Hotel North East,mohali punjab,Very Good,4.0,42,6115,72,1414,5,151,Punjab,zirakpur


In [22]:
df_hotels=df_hotels.drop(columns=["State_ID", "City_ID"])
df_hotels

,Hotel Name,Destination,Describe,Ratings,review_count,Actual Price,Offer Percentage,Price,State,City
0,Hotel O Sai President,Near Rawali Mandir,Good,3.5,453,3962,71,932,Uttar Pradesh,agra
1,Hotel O ASA AGRA,Plot number 1 Fatehabad road Plot number 1 Fat...,Excellent,4.5,275,2912,72,648,Uttar Pradesh,agra
2,Hotel O Salt And Pepper Hotel And Resturant,Near Prachin Pathwari Mandir,Good,3.5,137,4948,71,1137,Uttar Pradesh,agra
3,Hotel O KING PALACE,Near Fatehabad Road,Excellent,4.5,9,3378,71,797,Uttar Pradesh,agra
4,Hotel O 804621 Hotel Vintage House,1717 Fatehabad Road,Excellent,4.5,134,2912,70,702,Uttar Pradesh,agra
...,...,...,...,...,...,...,...,...,...,...
5257,Hotel O Elista,India - 140603,Very Good,4.0,138,4051,68,1036,Punjab,zirakpur
5258,Hotel O AV Continental,Near Chandigarh Airport Next to New Life Line ...,Fair,3.0,50,1797,62,542,Punjab,zirakpur
5259,Hotel O The Banur Heights,Mohali,Good,3.5,35,6552,77,1212,Punjab,zirakpur
5260,Collection O Hotel North East,mohali punjab,Very Good,4.0,42,6115,72,1414,Punjab,zirakpur
